# Compile machine scores

In [27]:
import os
import json
import constituency_retrieval_util

import pandas as pd

In [28]:
with open('data/moby-passages-36/item_to_level_to_constituency_ids.json') as f:
    item_to_level_to_constituency_ids = json.load(f)

In [29]:
ITEM_NUMBER = '330'
CHILD_PERFORMANCE_DIR = 'output/performance_matrix/330_child_updated_20200729'
GOLD_PERFORMANCE_DIR = 'output/performance_matrix/330_gold_updated_20200729'

In [12]:
session_to_child_df = {}
for file in os.listdir(CHILD_PERFORMANCE_DIR):
    session = file.split('.')[0]
    child_df = pd.read_csv(os.path.join(CHILD_PERFORMANCE_DIR, file), sep='\t')
    session_to_child_df[session] = child_df
    
session_to_gold_df = {}
for file in os.listdir(GOLD_PERFORMANCE_DIR):
    session = file.split('.')[0]
    gold_df = pd.read_csv(os.path.join(GOLD_PERFORMANCE_DIR, file), sep='\t')
    session_to_gold_df[session] = gold_df

In [31]:
for session, child_df in session_to_child_df.items():
    print(child_df)
    break

    Unnamed: 0    token  matches_expected  sframe  nframes  pitch_mean  \
0      <pause>  <pause>                 0       0       94 -131.728421   
1          sam      sam                 1      94       31   15.253494   
2      <pause>      NaN                 0       0        0    0.000000   
3          and      and                 1     125       25    5.681494   
4      <pause>      NaN                 0       0        0    0.000000   
..         ...      ...               ...     ...      ...         ...   
174    <pause>      NaN                 0       0        0    0.000000   
175      their    their                 1    5910       25    3.789494   
176    <pause>      NaN                 0       0        0    0.000000   
177     mother   mother                 1    5935      115  -63.676941   
178    <pause>  <pause>                 1    6050        3 -155.346506   

     pitch_start   pitch_end  pitch_high   pitch_low  pitch_slope  
0    -175.946506  -31.646506  -31.646506 -1